# Contract risk analysis

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [2]:
async Task<string> process_competion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Render an Scriban template

In [3]:
string render_tempate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

### Prepare a template and submit it for completion

In [4]:
var intent_template = @"System:
You are a travel assistant that can help determine intent. An intent is an action from the list of Defined Intents. An action is defined as a place, location, name, time or date.

Defined Intents:
GetItinerary
Reserve
CancelReservation
CheckReservation
GetReservation
GetWeather
Unknown

User:
What is the intent and entities for the following request:
{{input}}

Output format:
{
  ""intent"": ""intent"",
  ""entities"": [""action""]
}

Do not provide explanations. Output in JSON format only.";

In [9]:
var input="I want to make a reservation for 2 people at 7pm on Friday at Friday's Restaurant.";
var prompt = render_tempate(intent_template, new {input=input});
Console.WriteLine(await process_competion(prompt));

{
  "intent": "Reserve",
  "entities": ["reservation", "2 people", "7pm", "Friday", "Friday's Restaurant"]
}


In [10]:
input="What is the weather like in London?";
var prompt = render_tempate(intent_template, new {input=input});
Console.WriteLine(await process_competion(prompt));

{
  "intent": "GetWeather",
  "entities": ["London"]
}


In [11]:
input="What is the speed of light?";
var prompt = render_tempate(intent_template, new {input=input});
Console.WriteLine(await process_competion(prompt));

{
  "intent": "Unknown",
  "entities": []
}
